# `pickle` — General-Purpose Object Serialization

pickle serializes and deserializes Python objects into a byte stream.

**Key characteristics**

1. Supports most built-in and user-defined objects
2. Cross-version compatibility (best-effort)
3. Human-opaque binary format
4. Not secure against untrusted input
5. Typical use cases
6. Persisting Python objects
7. IPC between Python processes
8. Caching complex data structures

**Limitations**

1. Python-specific
2. Unsafe to unpickle untrusted data

In [21]:
import pickle

data = {"a": 1, "b": [1, 2, 3]}
blob = pickle.dumps(data)
restored = pickle.loads(blob)
restored

{'a': 1, 'b': [1, 2, 3]}

# `marshal` — Low-Level Internal Serialization

A CPython internal serialization format used mainly for .pyc files.

**Key characteristics**

1. Faster than pickle
2. Supports only simple built-in types
3. No backward compatibility guarantee
4. Not intended for general use
5. Typical use cases
6. Python internals
7. Compiler bytecode storage

**Limitations**

1. Version-dependent
2. Cannot serialize custom objects safely

In [22]:
import marshal

data = (1, 2, 3)
b = marshal.dumps(data)
marshal.loads(b)

(1, 2, 3)

# `copyreg` — Custom Pickling Rules

A helper module to control how objects are pickled.

**Key characteristics**

Extends pickle
Used for registering reduction functions
No storage by itself
Typical use cases
Pickling objects with special construction logic
Optimizing pickled representations

**Limitations**

Only meaningful when used with pickle

In [23]:
import copyreg, pickle


class Point:
    def __init__(self, x, y):
        self.x, self.y = x, y


def pickle_point(p):
    return Point, (p.x, p.y)


copyreg.pickle(Point, pickle_point)

pickle.dumps(Point(1, 2))

b'\x80\x05\x95\x1e\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\x05Point\x94\x93\x94K\x01K\x02\x86\x94R\x94.'

# `dbm` — Simple Key-Value Database (Bytes Only)

A low-level on-disk key-value store.

**Key characteristics**

1. Persistent storage
2. Keys and values must be bytes
3. Backend varies by OS (ndbm, gdbm, etc.)

**Typical use cases**

1. Lightweight persistent caches
2. Storing binary blobs

**Limitations**

1. No support for complex objects
2. Manual serialization required

In [24]:
import dbm

with dbm.open("data.db", "c") as db:
    db[b"user"] = b"Shravan"
    print(db), print(db.keys()), print(db["user"])

[b'user']
b'Shravan'


# shelve — Persistent Object Store (Pickle + dbm)

A dictionary-like interface combining pickle and dbm.

**Key characteristics**

1. Persistent storage for Python objects
2. Values are pickled automatically
3. Keys must be strings
4. Typical use cases
5. Simple object persistence
6. Prototyping and lightweight storage

**Limitations**

1. Performance overhead (pickle)
2. Unsafe for untrusted data
3. Not suitable for concurrent writes

In [25]:
import shelve

with shelve.open("store") as db:
    db["user"] = {"name": "Shravan", "age": 25}
    print(db), print(db["user"])

{'name': 'Shravan', 'age': 25}


| Module    | Purpose                  | Stores Objects | Persistent | Security | Stability    |
| --------- | ------------------------ | -------------- | ---------- | -------- | ------------ |
| `pickle`  | Serialize Python objects | Yes            | Optional   | ❌ Unsafe | Stable       |
| `marshal` | Internal serialization   | Limited        | Optional   | ❌ Unsafe | ❌ Unstable   |
| `copyreg` | Customize pickling       | N/A            | No         | Depends  | Stable       |
| `dbm`     | Key-value DB (bytes)     | No             | Yes        | ✔ Safe   | OS-dependent |
| `shelve`  | Persistent object store  | Yes            | Yes        | ❌ Unsafe | Stable       |

**When to Use What (Decision Guide)**

* Need to serialize Python objects? → `pickle`
* Need speed, internal tooling only? → `marshal`
* Custom pickling behavior? → `copyreg`
* Need persistent byte storage? → `dbm`
* Need persistent Python objects with dict API? → `shelve`

## `shelve = pickle (serialization) + dbm (storage)`
## `copyreg` modifies how pickle works
## `marshal` is for Python internals, not applications

In [26]:
import os

In [27]:
os.unlink('data.db')
os.unlink('store')